In [ ]:
import os
import tempfile
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Inicializar SparkSession
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
import time
time.sleep(1)
conf = SparkConf().setAppName("LimpiezaSilver").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

In [ ]:
# Configuración MinIO
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")
MINIO_BUCKET_BRONCE = os.environ.get("MINIO_BUCKET", "proyecto-bronze")
MINIO_BUCKET_SILVER = "proyecto-silver"

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

# Crear bucket Silver si no existe
try:
    minio_client.make_bucket(MINIO_BUCKET_SILVER)
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} creado')
except:
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} ya existe')

In [ ]:
# Cargar archivos Bronze
print("\n📥 CARGANDO DATOS DE BRONZE...")
archivo_reciente = None
try:
    print("Buscando archivos Bronze...")
    objects = minio_client.list_objects(MINIO_BUCKET_BRONCE, recursive=True)
    archivos_parquet = [obj.object_name for obj in objects if obj.object_name.endswith(".parquet")]
    
    if archivos_parquet:
        dfs = []
        for archivo in sorted(archivos_parquet)[-8:]:  # Últimos 8 archivos
            temp_dir = tempfile.gettempdir()
            temp_file = os.path.join(temp_dir, archivo.split("/")[-1])
            minio_client.fget_object(MINIO_BUCKET_BRONCE, archivo, temp_file)
            df_temp = spark.read.parquet(temp_file)
            dfs.append(df_temp)
            print(f"  Cargado: {archivo}")
        
        # Consolidar todos los DataFrames
        from functools import reduce
        spark_df = reduce(lambda x, y: x.union(y), dfs)
        print(f"\n✅ Datos consolidados: {spark_df.count()} registros")
    else:
        print("⚠️ Sin archivos en Bronze")
        spark_df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])
except Exception as e:
    print(f"⚠️ Error: {e}")
    spark_df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])

print(f"\n📊 DataFrame: {spark_df.count()} registros")

In [ ]:
# Iniciar limpieza
print("\n🧹 LIMPIEZA DE DATOS...")

initial_count = spark_df.count()

# 1. Eliminar duplicados exactos (solo si todos los valores son idénticos)
spark_df = spark_df.dropDuplicates()
duplicates_removed = initial_count - spark_df.count()
print(f"Duplicados exactos eliminados: {duplicates_removed}")
print(f"Registros mantenidos: {spark_df.count()}")

In [ ]:
# 2. Filtrar valores nulos críticos
spark_df = spark_df.filter(
    col('ip').isNotNull() & 
    col('temperature').isNotNull()
)
after_nulls = spark_df.count()
print(f"Después de filtrar nulos: {after_nulls} (eliminados: {duplicates_removed})")

# 3. Filtrar outliers (menos agresivo)
if 'temperature' in spark_df.columns and 'humidity' in spark_df.columns:
    temp_mean = spark_df.select('temperature').rdd.flatMap(lambda x: x).mean()
    temp_std = spark_df.select('temperature').rdd.flatMap(lambda x: x).stdev()
    
    if temp_mean is not None and temp_std is not None and temp_std > 0:
        # Hacer filtro menos estricto (5 desviaciones en lugar de 3)
        lower_bound = temp_mean - (5 * temp_std)
        upper_bound = temp_mean + (5 * temp_std)
        spark_df = spark_df.filter((col('temperature') >= lower_bound) & (col('temperature') <= upper_bound))

after_outliers = spark_df.count()
print(f"Outliers de temperatura eliminados: {after_nulls - after_outliers}")

# 4. Imputar valores nulos en columnas numéricas con la media
numeric_cols = ['temperature', 'humidity', 'pressure', 'pm25', 'light']
for col_name in numeric_cols:
    if col_name in spark_df.columns:
        mean_val = spark_df.filter(col(col_name).isNotNull()).select(col(col_name)).rdd.flatMap(lambda x: x).mean()
        if mean_val:
            spark_df = spark_df.fillna(mean_val, subset=[col_name])
            print(f"Valores nulos imputados en {col_name} con media: {mean_val:.2f}")

# 5. Enriquecimiento temporal (opcional)
if 'timestamp' in spark_df.columns:
    try:
        from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second
        spark_df = spark_df.withColumn("year", year(col("timestamp")))
        spark_df = spark_df.withColumn("month", month(col("timestamp")))
        spark_df = spark_df.withColumn("day", dayofmonth(col("timestamp")))
        spark_df = spark_df.withColumn("hour", hour(col("timestamp")))
    except:
        print("Enriquecimiento temporal skipeado (timestamp tiene formato complejo)")

print(f"\nLimpieza completada: {spark_df.count()} registros finales")

In [ ]:
# Guardar en Silver
print("\n💾 GUARDANDO EN SILVER...")

try:
    import io
    from pyspark.sql.types import TimestampType
    
    df_export = spark_df
    for field in spark_df.schema.fields:
        if isinstance(field.dataType, TimestampType):
            df_export = df_export.withColumn(field.name, col(field.name).cast("string"))
    
    pdf = df_export.toPandas()
    csv_buffer = pdf.to_csv(index=False)
    
    object_path = "sensor_data/cleaned_data.csv"
    csv_bytes = io.BytesIO(csv_buffer.encode('utf-8'))
    minio_client.put_object(
        MINIO_BUCKET_SILVER, 
        object_path, 
        csv_bytes,
        length=len(csv_buffer.encode('utf-8')),
        content_type="text/csv"
    )
    print(f"✅ Datos guardados: {object_path}")
    print(f"📊 Registros: {len(pdf)}")
    
except Exception as e:
    error_msg = str(e)
    if "UnsatisfiedLinkError" in error_msg or "NativeIO" in error_msg:
        print(f"⚠️ Warning Hadoop ignorado (no crítico)")
    else:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# Log de ejecución
print("\n📝 GUARDANDO LOG DE EJECUCIÓN...")

from datetime import datetime
import json

try:
    log_data = {
        'script': 'silver_data_cleaning',
        'execution_time': datetime.now().isoformat(),
        'records_processed': initial_count,
        'records_cleaned': spark_df.count(),
        'duplicates_removed': duplicates_removed,
        'output_file': 'cleaned_data.csv',
        'status': 'completed'
    }
    
    import pandas as pd
    log_df = pd.DataFrame([log_data])
    log_csv = log_df.to_csv(index=False)
    log_bytes = io.BytesIO(log_csv.encode('utf-8'))
    
    minio_client.put_object(
        MINIO_BUCKET_SILVER,
        "logs/execution_log.csv",
        log_bytes,
        length=len(log_csv.encode('utf-8')),
        content_type="text/csv"
    )
    
    print("✅ Log guardado exitosamente")
    print(f"\n{'='*70}")
    print("✅ LIMPIEZA DE SILVER COMPLETADA")
    print(f"{'='*70}")
    print(f"📍 Registros procesados: {initial_count}")
    print(f"📍 Registros limpios: {spark_df.count()}")
    print(f"📍 Duplicados removidos: {duplicates_removed}")
    print(f"📍 Porcentaje conservado: {((spark_df.count()/initial_count)*100):.1f}%")
    print(f"{'='*70}")
    
except Exception as e:
    print(f"Error al guardar log: {e}")

In [ ]:
# Limpiar SparkContext
print("\n🔚 CERRANDO SESIÓN...")

try:
    if 'spark' in locals():
        spark.stop()
        print("✅ Sesión de Spark cerrada correctamente")
except Exception as e:
    print(f"⚠️ Error al cerrar Spark: {e}")

In [ ]:
# Validación final de datos limpios
if spark_df:
    print("=== VALIDACIÓN FINAL ===")
    
    final_count = enriched_df.count()
    print(f"Total de registros finales: {final_count}")
    
    # Mostrar muestra de datos limpios
    print("\nMuestra de datos limpios:")
    enriched_df.select(
        'sensor_id', 'temperature', 'humidity', 'pressure', 
        'temperature_fahrenheit', 'comfort_index', 'data_quality_score'
    ).show(5)
    
    # Estadísticas de calidad
    print("\nDistribución de puntuación de calidad:")
    enriched_df.groupBy('data_quality_score').count().orderBy('data_quality_score').show()
    
    print("\nDistribución de índice de confort:")
    enriched_df.groupBy('comfort_index').count().show()
    
    # Estadísticas finales
    avg_quality = enriched_df.agg(avg('data_quality_score')).collect()[0][0]
    print(f"\nPuntuación promedio de calidad: {avg_quality:.2f}")

In [ ]:
# Guardar datos limpios en capa Silver
if spark_df:
    print("=== GUARDANDO EN CAPA SILVER (ARCHIVO ÚNICO) ===")
    
    # Convertir a Pandas para guardar en MinIO
    silver_df_pandas = enriched_df.toPandas()
    
    # Usar archivo único que se sobrescribe
    silver_path = f"{Config.SILVER_PATH}/cleaned_data.parquet"
    
    # Subir a MinIO (sobrescribir archivo existente)
    print(f"Sobrescribiendo archivo: {silver_path}")
    success = minio_client.upload_dataframe_as_parquet(silver_df_pandas, silver_path)
    
    if success:
        print(f"✅ Datos guardados exitosamente en capa Silver")
        print(f"  - Archivo: cleaned_data.parquet")
        print(f"  - Registros: {len(silver_df_pandas)}")
        print(f"  - Columnas: {len(silver_df_pandas.columns)}")
        
        # Mostrar archivos en Silver
        silver_files = minio_client.list_objects(Config.SILVER_PATH)
        print(f"\nArchivos totales en capa Silver: {len(silver_files)}")
        
        # Crear log de ejecución (archivo único que se sobrescribe)
        from datetime import datetime
        notebook_marker = {
            'notebook': 'silver_data_cleaning',
            'execution_time': datetime.now().isoformat(),
            'records_processed': len(silver_df_pandas),
            'output_file': 'cleaned_data.parquet',
            'status': 'completed'
        }
        
        log_df = pd.DataFrame([notebook_marker])
        log_path = f"{Config.SILVER_PATH}/execution_log.parquet"
        
        # Sobrescribir log de ejecución
        minio_client.upload_dataframe_as_parquet(log_df, log_path)
        print(f"📝 Log de ejecución actualizado")
        
    else:
        print("❌ Error al guardar en capa Silver")
else:
    print("No hay datos para procesar")

In [ ]:
# Limpieza final
print("=== LIMPIEZA FINAL ===")

# Cerrar sesión de Spark
if 'spark' in locals():
    spark.stop()
    print("✓ Sesión de Spark cerrada")

print("\n🎉 Proceso de limpieza completado exitosamente!")
print("\nLos datos limpios están disponibles en la capa Silver para")
print("su posterior procesamiento en la capa Gold (KPIs).")